# Loading the csv

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers

Functiom to add random noise

In [ ]:
def add_noise(image):
    # Convert image to numpy array
    img = np.array(image)
    # Generate random noise
    noise = np.random.randint(0, 255, img.shape)
    # Add noise to image
    noisy_image = np.clip(img + noise, 0, 255).astype(np.uint8)
    # Convert back to PIL Image
    return Image.fromarray(noisy_image)

Load the older csv and create a new csv

In [ ]:
# Path to the original CSV file
csv_path = '/content/drive/MyDrive/RAFdb/fer2013.csv'

# Path to the new CSV file
new_csv_path = '/content/drive/MyDrive/RAFdb/fer2013Augmented.csv'

Creating the dataframe

In [ ]:
# Load the original CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Create a new DataFrame for the noisy images
noisy_df = pd.DataFrame(columns=df.columns)


function to add noise

In [ ]:
for i in range(len(df)):
    # Load the image from the pixel values in the CSV file
    pixels = df.loc[i, 'pixels'].split()
    pixels = [int(p) for p in pixels]
    image = Image.fromarray(np.uint8(np.array(pixels).reshape(48,48)))
    # Add noise to the image
    noisy_image = add_noise(image)
    # Convert the noisy image back to pixel values and update the row
    noisy_pixels = np.array(noisy_image).reshape(-1).tolist()
    df.loc[i, 'pixels'] = ' '.join([str(p) for p in noisy_pixels])
    # Add the updated row to the new DataFrame
    noisy_df = noisy_df.append(df.loc[i])

Save new images

In [ ]:
noisy_df.to_csv(new_csv_path, index=False)

NameError: ignored

Display first twenty

In [ ]:
def display_first_20(df, title):
    fig, axs = plt.subplots(4, 5, figsize=(10, 10))
    for i in range(20):
        pixels = df.loc[i, 'pixels'].split()
        pixels = [int(p) for p in pixels]
        image = Image.fromarray(np.uint8(np.array(pixels).reshape(48,48)))
        row = i // 5
        col = i % 5
        axs[row, col].imshow(image, cmap='gray')
        axs[row, col].axis('off')
        axs[row, col].set_title(df.loc[i, 'emotion'])
    plt.suptitle(title)
    plt.show()

In [ ]:
# Display the first 20 images from the original CSV
display_first_20( df, 'Original Images')


In [ ]:
# Display the first 20 images from the new CSV
display_first_20(noisy_df, 'Noisy Images')

# Training

On OG data set.

In [ ]:
noisy_df = pd.read_csv(new_csv_path)

In [ ]:
def preprocess_images(images):
    images = np.array([np.fromstring(image, dtype=int, sep=' ').reshape(48, 48) for image in images])
    images = np.expand_dims(images, axis=-1)
    images = images / 255.0
    return images

function to train and test

In [ ]:
def train_model(data):
    # Preprocess the images and labels
    images = data['pixels']
    labels = data['emotion']
    images = preprocess_images(images)
    
    # Split the data into train/validation/test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.22, random_state=42)
    
    # Define the model architecture
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25)
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    print('Test accuracy:', accuracy)

In [ ]:
train_model(noisy_df)

Epoch 1/25
788/788 [==============================] - 17s 7ms/step - loss: 1.7497 - accuracy: 0.2855 - val_loss: 1.6416 - val_accuracy: 0.3624
Epoch 2/25
788/788 [==============================] - 5s 6ms/step - loss: 1.6235 - accuracy: 0.3634 - val_loss: 1.5744 - val_accuracy: 0.3918
Epoch 3/25
788/788 [==============================] - 4s 5ms/step - loss: 1.5670 - accuracy: 0.3891 - val_loss: 1.5561 - val_accuracy: 0.3988
Epoch 4/25
788/788 [==============================] - 5s 6ms/step - loss: 1.5206 - accuracy: 0.4082 - val_loss: 1.5130 - val_accuracy: 0.4074
Epoch 5/25
788/788 [==============================] - 5s 6ms/step - loss: 1.4777 - accuracy: 0.4281 - val_loss: 1.5038 - val_accuracy: 0.4213
Epoch 6/25
788/788 [==============================] - 4s 5ms/step - loss: 1.4340 - accuracy: 0.4424 - val_loss: 1.5030 - val_accuracy: 0.4229
Epoch 7/25
788/788 [==============================] - 5s 6ms/step - loss: 1.3932 - accuracy: 0.4632 - val_loss: 1.4989 - val_accuracy: 0.4237
Epoch

In [ ]:
train_model(df)

Epoch 1/25
788/788 [==============================] - 8s 7ms/step - loss: 1.6547 - accuracy: 0.3395 - val_loss: 1.4556 - val_accuracy: 0.4389
Epoch 2/25
788/788 [==============================] - 4s 5ms/step - loss: 1.3956 - accuracy: 0.4637 - val_loss: 1.3061 - val_accuracy: 0.4976
Epoch 3/25
788/788 [==============================] - 4s 5ms/step - loss: 1.2676 - accuracy: 0.5175 - val_loss: 1.2464 - val_accuracy: 0.5305
Epoch 4/25
788/788 [==============================] - 5s 6ms/step - loss: 1.1818 - accuracy: 0.5535 - val_loss: 1.2039 - val_accuracy: 0.5456
Epoch 5/25
788/788 [==============================] - 4s 5ms/step - loss: 1.1049 - accuracy: 0.5831 - val_loss: 1.2296 - val_accuracy: 0.5467
Epoch 6/25
788/788 [==============================] - 4s 5ms/step - loss: 1.0440 - accuracy: 0.6093 - val_loss: 1.1847 - val_accuracy: 0.5619
Epoch 7/25
788/788 [==============================] - 5s 7ms/step - loss: 0.9765 - accuracy: 0.6338 - val_loss: 1.1989 - val_accuracy: 0.5625
Epoch 